# 🚀 FINAL BOSS TRAINING: Fine-Tuning on Kaggle Data
**Goal:** Take your existing smart model (`roboflow_v2.pt`) and teach it "Warehouse Environments" using the huge Kaggle dataset.

**Strategy:** Transfer Learning (Fine-Tuning).

---

In [ ]:
# 1. UPLOAD YOUR MODEL
from google.colab import files
import os

print("⬆️ PLEASE UPLOAD 'roboflow_v2.pt' NOW...")
uploaded = files.upload()

# Verify upload
for filename in uploaded.keys():
    print(f"✅ RECEIVED: {filename}")
    # Rename to standard name for script
    os.rename(filename, 'roboflow_v2.pt')

In [ ]:
# 2. INSTALL & SETUP
%pip install ultralytics kagglehub -q
import shutil
import glob
import kagglehub
from ultralytics import YOLO

# Prepare Workspace
BASE_DIR = os.getcwd()
FINAL_DIR = os.path.join(BASE_DIR, "kaggle_dataset")
if os.path.exists(FINAL_DIR): shutil.rmtree(FINAL_DIR)

for split in ['train', 'valid']:
    os.makedirs(f"{FINAL_DIR}/{split}/images", exist_ok=True)
    os.makedirs(f"{FINAL_DIR}/{split}/labels", exist_ok=True)

print("✅ Environment Ready")

In [ ]:
# 3. DOWNLOAD KAGGLE DATA
try:
    print("⬇️ Downloading Kaggle Dataset...")
    k_path = kagglehub.dataset_download("zoya77/warehouse-delivery-box-detection-dataset")
    print(f"✅ Downloaded to: {k_path}")
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# 4. CRAWL & FORMAT (The 'Universal Crawler')
global_count = 0

print("🔄 Processing Kaggle Data...")
images = []
# Find all images recursively
for ext in ['*.jpg', '*.png', '*.jpeg']:
    images.extend(glob.glob(f"{k_path}/**/{ext}", recursive=True))

print(f"   Found {len(images)} images. converting...")

for img_path in images:
    # Find Label
    base = os.path.splitext(img_path)[0]
    txt_path = base + ".txt"
    
    # Check parallel 'labels' folder if not in same folder
    if not os.path.exists(txt_path):
        if "images" in txt_path: txt_path = txt_path.replace("images", "labels")
            
    if os.path.exists(txt_path):
        # Split 80/20
        split = 'valid' if (global_count % 5 == 0) else 'train'
        
        # Copy Image
        ext = os.path.splitext(img_path)[1]
        new_name = f"kaggle_{global_count:05d}"
        shutil.copy(img_path, f"{FINAL_DIR}/{split}/images/{new_name}{ext}")
        
        # Rewrite Label (Force Class 0)
        with open(txt_path, 'r') as f:
            lines = f.readlines()
        
        clean_lines = []
        for line in lines:
            parts = line.split()
            if len(parts) >= 5:
                # Replace class_id with 0
                clean_lines.append(f"0 {' '.join(parts[1:])}\n")
        
        with open(f"{FINAL_DIR}/{split}/labels/{new_name}.txt", 'w') as f:
            f.writelines(clean_lines)
            
        global_count += 1

print(f"✅ Processed {global_count} Valid Pairs")

# Create YAML
with open(f"{FINAL_DIR}/data.yaml", "w") as f:
    f.write(f"train: {FINAL_DIR}/train/images\nval: {FINAL_DIR}/valid/images\nnc: 1\nnames: ['cardboard_box']")

In [ ]:
# 5. FINE-TUNE (The Magic Step)
# Notice: model='roboflow_v2.pt' (NOT yolov8n.pt)

print("🚀 STARTING FINE-TUNING...")
model = YOLO("roboflow_v2.pt") 

model.train(
    data=f"{FINAL_DIR}/data.yaml",
    epochs=20,          # 20 is enough since it's already smart
    imgsz=640,
    batch=16,
    name="mega_model_final"
)

In [ ]:
# 6. DOWNLOAD THE CHAMPION
files.download('/content/runs/detect/mega_model_final/weights/best.pt')